# Homero to Robbe model results conversion

## Setup

In [9]:
# Set the code base path as the current working directory
import sys
sys.path.append('/workspaces/cvnlp_model_evaluation/src')

In [10]:
import pandas as pd
from os.path import join
import util.local_config as local_config
import json

In [11]:
# Get the homero results
homero_file_name = 'homero_results_1'
homero_results = pd.read_csv(join(local_config.TEXT_MODEL_RESULTS_FOLDER_PATH, 'homero_results', f'{homero_file_name}.csv'), index_col=0)

In [12]:
homero_results.head(5)

,Dialogue_ID,Utterance_ID,model_pred,0,1,2,3,4,5,6
0,0,0,6,0.078667,0.079547,0.127847,0.025739,0.047009,0.057820,0.583370
1,0,1,4,0.130414,0.091628,0.067096,0.246394,0.321644,0.093675,0.049150
2,0,2,4,0.128082,0.079608,0.102042,0.198844,0.351036,0.075768,0.064619
3,1,0,4,0.163778,0.108712,0.096479,0.210214,0.271940,0.106731,0.042146
4,1,1,2,0.164228,0.130154,0.266149,0.086425,0.136461,0.129400,0.087183


In [13]:
def convert_results_row(results_row):
    # Change the model prediction's one-hot index to our one-hot index
    results_row['model_pred'] = local_config.HOMER_CONVERSION_ONE_HOT_INDEX_TO_OUR_ONE_HOT_INDEX[results_row['model_pred']]

    # Get the model's prediction per emotion class
    neutral_prediction = results_row['4']
    surprise_prediction = results_row['6']
    fear_prediction = results_row['2']
    sadness_prediction = results_row['5']
    joy_prediction = results_row['3']
    disgust_prediction = results_row['1']
    anger_prediction = results_row['0']

    # Combine the model's prediction per emotion class into a list
    model_predictions_per_emotion_class = [
        neutral_prediction, 
        surprise_prediction, 
        fear_prediction, 
        sadness_prediction, 
        joy_prediction, 
        disgust_prediction, 
        anger_prediction
    ]

    # Get the json string representation of the model's prediction per emotion class
    model_predictions_per_emotion_class_json_string = json.dumps(model_predictions_per_emotion_class)

    # Add the model's prediction per emotion class to the results row
    results_row['model_outputs'] = model_predictions_per_emotion_class_json_string

    # Remove the model's prediction per emotion class columns from the results row
    results_row = results_row.drop(['4', '6', '2', '5', '3', '1', '0'])

    return results_row

## Main

In [14]:
# Convert the homero results
converted_homero_results = homero_results.apply(convert_results_row, axis=1)

# Rename the columns
converted_homero_results = converted_homero_results.rename({
        'Dialogue_ID': 'dialogue_id',
        'Utterance_ID': 'utterance_id',
        'model_pred': 'model_prediction'
    }, axis=1)

In [15]:
converted_homero_results.head(5)

,dialogue_id,utterance_id,model_prediction,model_outputs
0,0.0,0.0,1.0,"[0.047009233, 0.5833697, 0.12784746, 0.0578204..."
1,0.0,1.0,0.0,"[0.32164365, 0.04915007, 0.067095585, 0.093675..."
2,0.0,2.0,0.0,"[0.35103637, 0.06461884, 0.10204226, 0.0757681..."
3,1.0,0.0,0.0,"[0.27193958, 0.04214588, 0.09647946, 0.1067311..."
4,1.0,1.0,2.0,"[0.1364609, 0.08718251, 0.26614937, 0.12940022..."


In [16]:
# Save the converted homero results
converted_homero_results.to_csv(join(local_config.TEXT_MODEL_RESULTS_FOLDER_PATH, f'converted_{homero_file_name}.csv'), index=False)